<a href="https://colab.research.google.com/github/AkashCreator/Pyspark/blob/main/json_faltten.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=a44c74834c4fbdce51c8323c1decd700fb1ff6b19329d9a46f54bdd47ed725a2
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [3]:
spark=SparkSession.builder.appName('Flatten-Json').enableHiveSupport().getOrCreate()
spark

In [6]:
json_data=spark.read.json('./sample_data/newjson.json',multiLine=True)
json_data.show(truncate=False)

+--------------------------------------------------+------------------------------------------------------+---+--------+------------------------------------------------------------------------------------+
|address                                           |contacts                                              |id |name    |orders                                                                              |
+--------------------------------------------------+------------------------------------------------------+---+--------+------------------------------------------------------------------------------------+
|{New York, {40.7128, -74.006}, 123 Main St, 10001}|[{email, john.doe@example.com}, {phone, 555-123-4567}]|1  |John Doe|[{[{Widget, 2}, {Gadget, 1}], A123, 50.0}, {[{Widget, 3}, {Widget, 1}], B456, 75.0}]|
+--------------------------------------------------+------------------------------------------------------+---+--------+--------------------------------------------------------

In [15]:
json_data=json_data.withColumn('city',json_data.address.city)\
          .withColumn('street',json_data.address.street)\
          .withColumn('zipcode',json_data.address.zipcode)\
          .withColumn('latitude',json_data.address.coordinates.latitude)\
          .withColumn('longitude',json_data.address.coordinates.longitude)
json_data.show(truncate=False)

+--------------------------------------------------+------------------------------------------------------+---+--------+------------------------------------------------------------------------------------+--------+-----------+-------+--------+---------+
|address                                           |contacts                                              |id |name    |orders                                                                              |city    |street     |zipcode|latitude|longitude|
+--------------------------------------------------+------------------------------------------------------+---+--------+------------------------------------------------------------------------------------+--------+-----------+-------+--------+---------+
|{New York, {40.7128, -74.006}, 123 Main St, 10001}|[{email, john.doe@example.com}, {phone, 555-123-4567}]|1  |John Doe|[{[{Widget, 2}, {Gadget, 1}], A123, 50.0}, {[{Widget, 3}, {Widget, 1}], B456, 75.0}]|New York|123 Main St|10001  |40.7

In [16]:
json_data=json_data.withColumn('contacts',explode(json_data.contacts))
json_data.show(truncate=False)


+--------------------------------------------------+-----------------------------+---+--------+------------------------------------------------------------------------------------+--------+-----------+-------+--------+---------+
|address                                           |contacts                     |id |name    |orders                                                                              |city    |street     |zipcode|latitude|longitude|
+--------------------------------------------------+-----------------------------+---+--------+------------------------------------------------------------------------------------+--------+-----------+-------+--------+---------+
|{New York, {40.7128, -74.006}, 123 Main St, 10001}|{email, john.doe@example.com}|1  |John Doe|[{[{Widget, 2}, {Gadget, 1}], A123, 50.0}, {[{Widget, 3}, {Widget, 1}], B456, 75.0}]|New York|123 Main St|10001  |40.7128 |-74.006  |
|{New York, {40.7128, -74.006}, 123 Main St, 10001}|{phone, 555-123-4567}        |1 

In [17]:
json_data=json_data.withColumn('type',json_data.contacts.getItem('type'))\
                    .withColumn('value',json_data.contacts.getItem('value'))
json_data.show(truncate=False)

+--------------------------------------------------+-----------------------------+---+--------+------------------------------------------------------------------------------------+--------+-----------+-------+--------+---------+-----+--------------------+
|address                                           |contacts                     |id |name    |orders                                                                              |city    |street     |zipcode|latitude|longitude|type |value               |
+--------------------------------------------------+-----------------------------+---+--------+------------------------------------------------------------------------------------+--------+-----------+-------+--------+---------+-----+--------------------+
|{New York, {40.7128, -74.006}, 123 Main St, 10001}|{email, john.doe@example.com}|1  |John Doe|[{[{Widget, 2}, {Gadget, 1}], A123, 50.0}, {[{Widget, 3}, {Widget, 1}], B456, 75.0}]|New York|123 Main St|10001  |40.7128 |-74.006  |emai

In [18]:
json_data.dtypes

[('address',
  'struct<city:string,coordinates:struct<latitude:double,longitude:double>,street:string,zipcode:string>'),
 ('contacts', 'struct<type:string,value:string>'),
 ('id', 'bigint'),
 ('name', 'string'),
 ('orders',
  'array<struct<items:array<struct<product:string,quantity:bigint>>,order_id:string,total:double>>'),
 ('city', 'string'),
 ('street', 'string'),
 ('zipcode', 'string'),
 ('latitude', 'double'),
 ('longitude', 'double'),
 ('type', 'string'),
 ('value', 'string')]

In [20]:
json_data=json_data.withColumn('Orders',explode(json_data.orders))
json_data.show(truncate=False)

+--------------------------------------------------+-----------------------------+---+--------+----------------------------------------+--------+-----------+-------+--------+---------+-----+--------------------+
|address                                           |contacts                     |id |name    |Orders                                  |city    |street     |zipcode|latitude|longitude|type |value               |
+--------------------------------------------------+-----------------------------+---+--------+----------------------------------------+--------+-----------+-------+--------+---------+-----+--------------------+
|{New York, {40.7128, -74.006}, 123 Main St, 10001}|{email, john.doe@example.com}|1  |John Doe|{[{Widget, 2}, {Gadget, 1}], A123, 50.0}|New York|123 Main St|10001  |40.7128 |-74.006  |email|john.doe@example.com|
|{New York, {40.7128, -74.006}, 123 Main St, 10001}|{email, john.doe@example.com}|1  |John Doe|{[{Widget, 3}, {Widget, 1}], B456, 75.0}|New York|123 Mai

In [22]:
json_data=json_data.withColumn('order_id',json_data.Orders.getItem('order_id'))\
                    .withColumn('total',json_data.Orders.getItem('total'))\
                    .withColumn('items',json_data.Orders.getItem('items'))
json_data.show(truncate=False)

+--------------------------------------------------+-----------------------------+---+--------+----------------------------------------+--------+-----------+-------+--------+---------+-----+--------------------+--------+-----+--------------------------+
|address                                           |contacts                     |id |name    |Orders                                  |city    |street     |zipcode|latitude|longitude|type |value               |order_id|total|items                     |
+--------------------------------------------------+-----------------------------+---+--------+----------------------------------------+--------+-----------+-------+--------+---------+-----+--------------------+--------+-----+--------------------------+
|{New York, {40.7128, -74.006}, 123 Main St, 10001}|{email, john.doe@example.com}|1  |John Doe|{[{Widget, 2}, {Gadget, 1}], A123, 50.0}|New York|123 Main St|10001  |40.7128 |-74.006  |email|john.doe@example.com|A123    |50.0 |[{Widget, 2}

In [24]:
json_data=json_data.withColumn('items',explode(json_data.Orders.items))

In [25]:
json_data.show(truncate=False)

+--------------------------------------------------+-----------------------------+---+--------+----------------------------------------+--------+-----------+-------+--------+---------+-----+--------------------+--------+-----+-----------+
|address                                           |contacts                     |id |name    |Orders                                  |city    |street     |zipcode|latitude|longitude|type |value               |order_id|total|items      |
+--------------------------------------------------+-----------------------------+---+--------+----------------------------------------+--------+-----------+-------+--------+---------+-----+--------------------+--------+-----+-----------+
|{New York, {40.7128, -74.006}, 123 Main St, 10001}|{email, john.doe@example.com}|1  |John Doe|{[{Widget, 2}, {Gadget, 1}], A123, 50.0}|New York|123 Main St|10001  |40.7128 |-74.006  |email|john.doe@example.com|A123    |50.0 |{Widget, 2}|
|{New York, {40.7128, -74.006}, 123 Main St,

In [26]:
json_data=json_data.withColumn('product',json_data.items.getItem('product'))\
                    .withColumn('quantity',json_data.items.getItem('quantity'))
json_data.show(truncate=False)

+--------------------------------------------------+-----------------------------+---+--------+----------------------------------------+--------+-----------+-------+--------+---------+-----+--------------------+--------+-----+-----------+-------+--------+
|address                                           |contacts                     |id |name    |Orders                                  |city    |street     |zipcode|latitude|longitude|type |value               |order_id|total|items      |product|quantity|
+--------------------------------------------------+-----------------------------+---+--------+----------------------------------------+--------+-----------+-------+--------+---------+-----+--------------------+--------+-----+-----------+-------+--------+
|{New York, {40.7128, -74.006}, 123 Main St, 10001}|{email, john.doe@example.com}|1  |John Doe|{[{Widget, 2}, {Gadget, 1}], A123, 50.0}|New York|123 Main St|10001  |40.7128 |-74.006  |email|john.doe@example.com|A123    |50.0 |{Widge

In [27]:
json_data=json_data.select('id','name','city','street','zipcode','latitude','longitude','type','value','order_id',\
                           'total','product','quantity')

In [28]:
json_data.show()

+---+--------+--------+-----------+-------+--------+---------+-----+--------------------+--------+-----+-------+--------+
| id|    name|    city|     street|zipcode|latitude|longitude| type|               value|order_id|total|product|quantity|
+---+--------+--------+-----------+-------+--------+---------+-----+--------------------+--------+-----+-------+--------+
|  1|John Doe|New York|123 Main St|  10001| 40.7128|  -74.006|email|john.doe@example.com|    A123| 50.0| Widget|       2|
|  1|John Doe|New York|123 Main St|  10001| 40.7128|  -74.006|email|john.doe@example.com|    A123| 50.0| Gadget|       1|
|  1|John Doe|New York|123 Main St|  10001| 40.7128|  -74.006|email|john.doe@example.com|    B456| 75.0| Widget|       3|
|  1|John Doe|New York|123 Main St|  10001| 40.7128|  -74.006|email|john.doe@example.com|    B456| 75.0| Widget|       1|
|  1|John Doe|New York|123 Main St|  10001| 40.7128|  -74.006|phone|        555-123-4567|    A123| 50.0| Widget|       2|
|  1|John Doe|New York|1